In [6]:
!pip install unrar
!pip install rarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [123]:
import pandas as pd
import numpy as np
import re
import rarfile
import zipfile
import codecs
import pickle
from sklearn.model_selection import train_test_split

In [8]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## **Upload Hamshahri file from google drive**

In [57]:
! gdown --id 1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D3yt99D0GcCRCbdKbUQGxbqjkeh91hTg
To: /content/hamshahri.rar
100% 873M/873M [00:18<00:00, 48.3MB/s]


In [58]:
def extract_rar_file(path, extract_path):
  files = []
  with rarfile.RarFile(path, 'r') as rf:
    files = rf.namelist()
    rf.extractall(extract_path)
    
  for file in files:
    path_to_file = fr"{extract_path}\{file}"  

In [59]:
def extract_zip_file(path, extract_path):
  with zipfile.ZipFile(path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

extract hamshahri file

In [60]:
path = 'hamshahri.rar'
extract_path = 'hamshahri'
extract_rar_file(path, extract_path)

extract hamshahri-corpus file

In [61]:
hamshahri_corpus_path = 'hamshahri/hamshahriold/Corpus/Hamshahri-Corpus.zip'
hamshahri_corpus_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus'
extract_zip_file(hamshahri_corpus_path, hamshahri_corpus_extract_path)
HAMSHAHRI_CORPUS_PATH = 'hamshahri/hamshahriold/Corpus/hamshahri-corpus/Hamshahri-Corpus.txt'

extract stopwords file

In [67]:
hamshahri_stopwords_path = 'hamshahri/hamshahriold/Corpus/PersianStopWords.zip'
hamshahri_stopwords_extract_path = 'hamshahri/hamshahriold/Corpus/hamshahri-stopwords'
extract_zip_file(hamshahri_stopwords_path, hamshahri_stopwords_extract_path)
HAMSHAHRI_STOPWORDS_PATH = '/content/hamshahri/hamshahriold/Corpus/hamshahri-stopwords/PersianStopWords.txt'

### **Read Hamshahri Corpus**

In [63]:
%%time
with codecs.open(HAMSHAHRI_CORPUS_PATH, "r", "utf-8") as text:
    sentences = []
    pattern = re.compile(".Cat")
    s = ' '
    for line in text:
      for match in re.finditer(pattern, line):
        sentences.append((line,s))
        s = ' '
      s += line         

CPU times: user 49 s, sys: 2.66 s, total: 51.7 s
Wall time: 50.4 s


In [ ]:
category_punctuation = [re.sub('\.Cat\t', '', line[0]) for line in sentences]
sentences_punctuation = [line[1].replace('\n', '') for line in sentences] 
sentences_punctuation = [line.replace('\r', '') for line in sentences_punctuation] 
sentences_punctuation = [re.sub('\.Cat\s\w+', '', sentence) for sentence in sentences_punctuation]
sentences_punctuation = [re.sub('\.DID\\t\d+S\d+.Date\\t\d+\\\\\d+\\\\\d+', '', sentence) for sentence in sentences_punctuation]

In [ ]:
print(sentences_punctuation[1])

  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين ن

In [ ]:
corpus = []
for i in range(len(sentences)):
  corpus.append({'text': sentences_punctuation[i], 'category': category_punctuation[i-1]})

In [ ]:
corpus[1]

{'category': 'adabh\r\n',
 'text': '  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پاي

In [ ]:
uncleaned_data = pd.DataFrame(corpus[1:], columns=['text', 'category'])

### **Read StopWords**

In [68]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [69]:
stopwords = read_stop_words(HAMSHAHRI_STOPWORDS_PATH)
print(len(stopwords))

796


In [70]:
# install hazm library
!pip install hazm
from hazm import word_tokenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
corpus = uncleaned_data['text'].values

In [ ]:
corpus[0]

'  در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين 

In [ ]:
word_tokenized = [word_tokenize(t) for t in corpus]

In [ ]:
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [ ]:
del corpus
del stopwords
del word_tokenized

In [ ]:
corpus = []
for sentence in word_tokenized_filtered:
  s = ' '
  for w in sentence:
    s += w + ' '
  corpus.append(s)  

In [ ]:
del word_tokenized_filtered

In [ ]:
data = pd.DataFrame(columns=['text', 'category'])

### **Crate DataFrame**

In [ ]:
data['text'] = corpus
data['category'] = uncleaned_data['category'].str.replace('\\r\\n','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [174]:
data.head()

,text,category
0,زندگي گروهي طريق هنر نگاهي نمايشگاه آثار هنري...,adabh
1,هنري جهان نمايشگاه هنر خدمت ديكتاتورها نمايشگ...,adabh
2,نگارخانه گالري گلستان نمايشگاه طرح ساخت نقره ...,adabh
3,جدي بگيريم مطالعه مقدماتي نقش بازي زندگي اجتم...,ejtem
4,سياه غباري سترده اشاره; رغم گسترش توسعه روزاف...,elmfa


In [ ]:
len(data)

165261

### **Export DataFrame to CSV**

In [ ]:
data.to_csv('NEWS_dataset.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/NEWS_dataset.csv" "/content/drive/MyDrive/NEWS_dataset.csv"

Mounted at /content/drive


# **Read Dataset**

In [1]:
! gdown --id 1sv5Bcxg6AwLYSkeK8LwsXUSRLJDxn5xD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1sv5Bcxg6AwLYSkeK8LwsXUSRLJDxn5xD
To: /content/NEWS_dataset.csv
100% 426M/426M [00:04<00:00, 89.1MB/s]


In [189]:
data = pd.read_csv('NEWS_dataset.csv')

In [190]:
data = data[data['text'].str.len() <= 10_000]

In [191]:
data.head()

,text,category
0,زندگي گروهي طريق هنر نگاهي نمايشگاه آثار هنري...,adabh
1,هنري جهان نمايشگاه هنر خدمت ديكتاتورها نمايشگ...,adabh
2,نگارخانه گالري گلستان نمايشگاه طرح ساخت نقره ...,adabh
3,جدي بگيريم مطالعه مقدماتي نقش بازي زندگي اجتم...,ejtem
4,سياه غباري سترده اشاره; رغم گسترش توسعه روزاف...,elmfa


In [192]:
len(data)

164740

In [193]:
data = data[data.groupby('category')['category'].transform('count') > 500].copy()

In [194]:
num_unique_categories = data['category'].nunique()

In [136]:
num_unique_categories

32

In [137]:
len(data)

155603

In [138]:
data['category'].value_counts()

siasi    17462
eqtes    16654
kharj    14664
akhar    13811
vrzsh    13009
shari    10130
elmif     8865
shahr     8398
shrst     8248
ejtem     8245
gungn     7535
adabh     5164
havad     3757
elmfa     1781
gozar     1418
maqal     1411
polig     1370
aeqts     1347
soxan     1326
nnaft     1220
jvarz     1179
econw     1011
artw       973
mohit      950
sporw      947
intep      769
nameh      762
bankb      759
shora      647
Cultw      634
santj      612
maref      545
Name: category, dtype: int64

**One Hot the Y**

In [160]:
Y_one_hot = pd.get_dummies(data['category'])

In [161]:
category_types = Y_one_hot.columns.values

In [162]:
del Y_one_hot

In [163]:
category_types

array(['Cultw', 'adabh', 'aeqts', 'akhar', 'artw', 'bankb', 'econw',
       'ejtem', 'elmfa', 'elmif', 'eqtes', 'gozar', 'gungn', 'havad',
       'intep', 'jvarz', 'kharj', 'maqal', 'maref', 'mohit', 'nameh',
       'nnaft', 'polig', 'santj', 'shahr', 'shari', 'shora', 'shrst',
       'siasi', 'soxan', 'sporw', 'vrzsh'], dtype=object)

In [144]:
Y = pd.get_dummies(data['category']).values

In [145]:
Y[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [20]:
list(data['text'])[0]

' زندگي گروهي طريق هنر نگاهي نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان باطن ظاهري انسانهاهستند هيبتي حشره تابلوهاي نقاشي نمايشگر گوشه زندگي مورچه ديده ايم سنگين خودرا دوش كشد راه خسته نتيجه آمدن حادثه اتفاقي دستش رها گريز خطر گذارد فرار ساعتي نقطه برمي گردد دوش كشيده نفس زنان عرق ريزان منزل رساند داند راه رفت مورچه استعداد غريزه عملي شروع پايان رساند داند نسل آينده مزاياي استفاده نمايد داستان درازي انتها پاياني ميليونها سال گذشته ميلياردها سال خواهد_گذشت وهمين برنامه ميليونها تجديد گفت سير حيات تكامل پايان ناپذير اسرار جهان پاياني بخواهيم چيز پايان ناپذير برسيم انتهايي نيست جايي رسيم كتاب مورچگان نوشته موريس مترلينگ مورچگان تابلوهاي نقاشي احمد طباطبايي جايگاه ويژه دارند نمايشگاهي آثار هنرمند اول 23 خرداد نگارخانه برگ برپا 36 اثر موضوع نمايش عمومي درآمد طباطبايي مدت 18 سالي نقاشي پردازد موضوع مورچگان كار علت انتخاب سوژه گويد روزهاي زندگي غمگين بودم زماني خواب رفتم مورچه ديدم تصميم گرفتم طرح بكشم كار نظرم جالب آمد روز طراحي ادامه دادم نقاشي مدلها تغييراتي ايجاد ضمن حشر

**text tokenizer**

In [21]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(data['text']))

In [23]:
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [24]:
vocab_size = len(tokenizer.word_index)+1

In [25]:
data_to_sequences = tokenizer.texts_to_sequences(list(data['text']))
maxlen = max_length(data_to_sequences)

In [26]:
maxlen

1843

In [27]:
pad_data_to_sequences = pad_sequences(data_to_sequences, maxlen=maxlen)

In [28]:
del data
del data_to_sequences

In [29]:
X_train, X_test, y_train, y_test = train_test_split(pad_data_to_sequences, Y, test_size=0.3, random_state=42)

In [30]:
del pad_data_to_sequences

In [31]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(108922, 1843)
(46681, 1843)
(108922, 32)
(46681, 32)


### **Neural Network**

In [32]:
embedding_dim = 50

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_unique_categories, activation='softmax'))

In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1843, 50)          24286200  
                                                                 
 bidirectional (Bidirectiona  (None, 1843, 400)        401600    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 400)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 200)               80200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0

In [36]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)

Epoch 1/5
426/426 [==============================] - 372s 854ms/step - loss: 2.0306 - accuracy: 0.4086 - val_loss: 1.1344 - val_accuracy: 0.6668
Epoch 2/5
426/426 [==============================] - 374s 878ms/step - loss: 0.9946 - accuracy: 0.7205 - val_loss: 0.8412 - val_accuracy: 0.7553
Epoch 3/5
426/426 [==============================] - 371s 871ms/step - loss: 0.7142 - accuracy: 0.7931 - val_loss: 0.8015 - val_accuracy: 0.7700
Epoch 4/5
426/426 [==============================] - 368s 864ms/step - loss: 0.5378 - accuracy: 0.8414 - val_loss: 0.8296 - val_accuracy: 0.7744
Epoch 5/5
426/426 [==============================] - 370s 870ms/step - loss: 0.3987 - accuracy: 0.8821 - val_loss: 0.9002 - val_accuracy: 0.7664


In [37]:
model.save('model.h5')

### **Upload tokenizer and model to google drive**

In [38]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "/content/tokenizer.h5" "/content/drive/MyDrive/news_classification/tokenizer.h5"
!cp -r "/content/model.h5" "/content/drive/MyDrive/news_classification/model.h5"

Mounted at /content/drive


### **Read tokenizer and model**

In [39]:
! gdown --id 1gnbOKJB-KePiOcDH3glTo85Vw2nUYk_7
! gdown --id 1IHM5DYxWoujkSHOdF_abRqmS_tM6TAdZ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1gnbOKJB-KePiOcDH3glTo85Vw2nUYk_7
To: /content/tokenizer.h5
100% 28.8M/28.8M [00:00<00:00, 137MB/s] 
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1IHM5DYxWoujkSHOdF_abRqmS_tM6TAdZ
To: /content/model.h5
100% 297M/297M [00:01<00:00, 237MB/s]


In [40]:
# laod model
model = load_model('model.h5')

In [104]:
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [41]:
model

In [170]:
def preprocess(text, stopwords):
  word_tokenized = word_tokenize(text) 
  word_tokenized_filtered = [w for w in word_tokenized if w not in stopwords]
  cleaned_text = ' '.join(map(str, word_tokenized_filtered)) 
  return cleaned_text

In [171]:
MAX_LENGTH = 1843
def prediction(text):
  cleaned_text = preprocess(text, stopwords)
  text_to_sequences = tokenizer.texts_to_sequences([cleaned_text])
  pad_text_to_sequences = pad_sequences(text_to_sequences, maxlen=MAX_LENGTH)
  category = model.predict([pad_text_to_sequences])[0]
  category_index = np.where(category == max(category))
  return category_types[category_index]

In [172]:
example_text = "در زندگي گروهي از طريق هنر نگاهي به نمايشگاه آثار هنري احمد طباطبايي موضوع آثار طباطبايي مورچگان هستند ولي در باطن چنين ظاهري، اين انسانهاهستند كه در هيبتي حشره گونه در تابلوهاي نقاشي نمايشگر گوشه هايي از زندگي خود هستند. مورچه اي را ديده ايم كه بار سنگين خودرا به دوش مي كشد و در بين راه خسته مي شود يا در نتيجه پيش آمدن يك حادثه اتفاقي بار از دستش رها شده و براي گريز از خطر آن را مي گذارد و فرار مي كند. اما ساعتي بعد به همان نقطه برمي گردد و بار را دوباره به دوش كشيده نفس زنان و عرق ريزان آن را به منزل او مي رساند مي داند كه اگر در ميان راه از بين رفت، يك مورچه ديگر با همان استعداد غريزه عملي را كه او شروع كرده به پايان مي رساند. پس او مي داند كه بعد از خودش نسل آينده اي وجود دارد كه بايد از مزاياي آن استفاده نمايد. چه داستان درازي، انتها و پاياني هم ميليونها ندارد سال گذشته، ميلياردها سال ديگر خواهد گذشت وهمين برنامه ميليونها بار ديگر تجديد مي شود. پس بايد گفت سير حيات و تكامل پايان ناپذير است. اسرار جهان پاياني ندارد. پس اگر بخواهيم به يك چيز پايان ناپذير برسيم چون انتهايي در بين نيست، به جايي نمي رسيم. (از كتاب مورچگان، نوشته موريس مترلينگ )مورچگان در تابلوهاي نقاشي احمد طباطبايي جايگاه ويژه اي دارند. در نمايشگاهي كه از آثار اين هنرمند (اول تا 23 خرداد ) در نگارخانه برگ برپا شد 36 اثر با اين موضوع به نمايش عمومي درآمد. طباطبايي در مدت 18 سالي كه به نقاشي مي پردازد همواره بر روي موضوع مورچگان كار كرده است. وي درباره علت انتخاب چنين سوژه اي مي گويد: در يكي از روزهاي زندگي خود بسيار غمگين بودم و زماني كه به خواب رفتم، مورچه اي را ديدم. سپس تصميم گرفتم طرح آن را بكشم. اين كار به نظرم جالب آمد و از آن روز به بعد اين نوع طراحي را ادامه دادم كه در هر بار نقاشي در اين مدلها تغييراتي را ايجاد مي كنم، ضمن اينكه اين حشره به دليل زندگي منظمي كه دارد، برايم جذاب است. نظم يكي از اركان آثار نقاشي طباطبايي است. عليرغم موضوعات ساده، تركيببندي دو بعدي و تنوع كم در رنگ آميزي سطوح، استحكام طرحها و نقوش بر روي پرده اثر نشان از نظمي دروني دارد. اين نظم خود را در آناتومي و اندام مورچگان به نوع ديگري نشان مي دهد. مشخصه اصلي مورچگان دست و پا و تقسيم بندي سه گانه بدن آنها است و مهمتر آنكه روي شش دست و پا راه مي روند. اما در نقاشيهاي طباطبايي آنها روي دو پا مثل انسان مي ايستند. سرهايشان بزرگترين اندام بدنشان است و چشمها به طور اغراق آميزي در سرجا گرفته است. هدف نقاش از اين تغييرات در اندام مورچگان؟ چيست باآنكه خود نقاش گفته و از نقاشيها استنباط  مي شود، موضوع آثار طباطبايي مورچگان است اما در باطن چنين ظاهري، اين انسانها هستند كه در هيبتي حشره گونه در تابلوهاي نقاش نمايشگر گوشه هايي از زندگي خود هستند. آنها با اندام سياه خود در زمينه اي خاكستري بادكنكهاي رنگي به دست دارند و عروسي مي كنند. تغيير در اندام مورچگان باعث شده چهره و حالات سر و صورت تنها اندام بازگو كننده احساسات موضوع در تابلوي نقاشي نباشد بلكه تمام بدن اين  مورچه - انسانهاحسي را پديد مي آورند كه در تابلو سايه مي اندازد. احساساتي نظير به دام افتادن در دام يك عنكبوت، مادري كردن، تفاهم و دوستي. نكته ديگر، نقاشي درختان به صورت اسليمي هاي ايراني در حاشيه تابلوها با خطوط و رنگهايي زنده و شاداب است. اين نقشها در تقابل بصري با  مورچه - انسانهاست اما در يك هماهنگي دروني با مفهوم مورچه - انسانها به تصوير كشيده شده اند."

In [179]:
prediction(example_text)

array(['adabh'], dtype=object)

In [196]:
prediction(data['text'][2])

array(['adabh'], dtype=object)

In [197]:
prediction(data['text'][3])

array(['ejtem'], dtype=object)

In [205]:
prediction(data['text'][5])

array(['elmfa'], dtype=object)